# Real World Applications

## Identifying academically vulnerable learners in first-year science programmes 

In [day05a-vulnerable-learners.ipynb](day05a-vulnerable-learners.ipynb), we
train (propositional) symbolic models aiming to identify academically vulnerable
learners in first-year science programmes at a South African higher-education
institution.

This is based on the paper:
[
    Identifying academically vulnerable learners in first-year science
    programmes at a South African higher-education institution
](
    https://sacj.cs.uct.ac.za/index.php/sacj/article/view/832
)

## Interpretable land cover classification with modal decision trees (extra)

In [
    day05b-land-cover-classification.ipynb
](
    day05b-land-cover-classification.ipynb
), you can find an example of (multi-modal) symbolic learning models trained to
classify land coverage, a segmentation problem aiming to recognize what can be
found in a satellite image.

This is based on the paper: 
[
    Interpretable land cover classification with modal decision trees
](
    https://www.tandfonline.com/doi/pdf/10.1080/22797254.2023.2262738
)

To run this notebook, you first need to download the following datasets and
place them in the `/datasets/paviaU` folder:
- [Pavia University](https://www.ehu.eus/ccwintco/uploads/e/ee/PaviaU.mat)
- [Pavia University GT](https://www.ehu.eus/ccwintco/uploads/5/50/PaviaU_gt.mat)
